In [4]:
#importacion de librerias
import pandas as pd
import numpy as np
import json
import ast
import datetime as dt
import os.path


## Extraccion de Datos

los archivos de los datos vienen en archivos .json y vienen anidados

De siguiente link se pueden descargar los datasets originales: https://drive.google.com/drive/folders/1XrgahQxKIrEwvViS9NzGbTVBnPkQ-2EW?usp=sharing

### Archivos games

In [5]:
#Ingesta del archivo games.json
games = [] 

with open(r'C:\Users\jrgui\Documents\Data PI1\output_steam_games.json') as file:
    for line in file: 
        data = json.loads(line)
        games.append(data)


df_games = pd.DataFrame(games)
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Archivos Reviews

In [3]:
#Ingesta del archivo reviews.json

reviews = []
with open(r'C:\Users\jrgui\Documents\Data PI1\australian_user_reviews.json', 'r', encoding = 'utf-8') as file:
    for line in file.readlines():
        reviews.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(reviews)
df_reviews.head(2)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."


In [86]:
#Desanidar el 2do nivel reviews

dfreviews_exp = df_reviews.explode('reviews')
dfreviews_nor = pd.json_normalize(dfreviews_exp)
dfreviews_nor = pd.json_normalize(dfreviews_exp['reviews'])
dfreviews_exp.reset_index(drop=True, inplace=True)
dfreviews_nor.reset_index(drop=True, inplace=True)

df_reviews = pd.concat([dfreviews_exp.drop('reviews', axis=1), dfreviews_nor], axis=1)
df_reviews.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.


### Archivos Items

In [87]:
#Ingesta del archivo items.json, falta desanidar el 2do nivel de  anidacion

items = []
with open(r'C:\Users\jrgui\Documents\Data PI1\australian_users_items.json', 'r', encoding = 'utf-8') as file:
    for line in file.readlines():
        items.append(ast.literal_eval(line))

df_items = pd.DataFrame(items)
df_items.head(2)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [88]:
#Desanidar el 2do nivel items

dfitems_exp = df_items.explode('items')
dfitems_nor = pd.json_normalize(dfitems_exp['items'])
dfitems_exp.reset_index(drop=True, inplace=True)
dfitems_nor.reset_index(drop=True, inplace=True)

df_items = pd.concat([df_items.drop('items', axis=1), dfitems_nor], axis=1)
df_items.head(2)

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277.0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,js41637,888.0,76561198035864385,http://steamcommunity.com/id/js41637,20,Team Fortress Classic,0.0,0.0


## Limpieza General

In [89]:
#Borrar los Nan y null 
df_games.dropna(inplace=True)
df_reviews.dropna(inplace=True)
df_items.dropna(inplace=True)

#Reinicia el indice por defecto de pandas
df_games.reset_index(drop=True, inplace=True)
df_reviews.reset_index(drop=True, inplace=True)
df_items.reset_index(drop=True, inplace=True)

## Transformación
 Se crea un dataframe para la mayoria de las funciones

In [90]:
#Se une el dataframe de games(id) con el de items(item_id)
df1_2 = pd.merge(df_games, df_reviews, how = 'right', left_on ='id', right_on='item_id')
df1_2.columns
df1_2.drop(['publisher', 'app_name', 'title', 'url', 'last_edited',
       'tags', 'reviews_url', 'specs', 'early_access', 'id', 'user_url', 'funny', 'helpful'] , axis=1, inplace=True)

In [91]:
df1_2.head(1)


,genres,release_date,price,developer,user_id,posted,item_id,recommend,review
0,[Action],2009-05-14,19.99,Tripwire Interactive,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...


In [92]:
#Se une el dataframe anterior con el dataframe_items, para hacer el dataframe unico con todos los datos necesarios
df = pd.merge(df1_2, df_items, how = 'left', left_on ='user_id', right_on='user_id')

In [93]:
df.columns

Index(['genres', 'release_date', 'price', 'developer', 'user_id', 'posted',
       'item_id_x', 'recommend', 'review', 'items_count', 'steam_id',
       'user_url', 'item_id_y', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')

In [94]:
df.drop(['steam_id','items_count','user_url','playtime_2weeks', 'item_id_y'] , axis=1, inplace=True)


In [95]:
df.head(1)

,genres,release_date,price,developer,user_id,posted,item_id_x,recommend,review,item_name,playtime_forever
0,[Action],2009-05-14,19.99,Tripwire Interactive,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,Counter-Strike,6.0


#### Se crea una nueva columna **release_year**, solo con el valor del año y se borra **release_date** 

In [96]:
#Transforma a la columna release_date a datatime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

#Crea una nueva columna con el valor de año de release_date
df["release_year"] = (df["release_date"].dt.strftime("%Y"))

#Cambia la columna release_year a int
df.dropna(inplace=True)
df["release_year"] = df["release_year"].astype('int')#, errors="ignore")

#Borra la columna release_date, ya que no sera necesaria
df.drop(['release_date'] , axis=1, inplace=True)


#### Se crea una nueva columna posted_year, con el valor del añio y se borra **posted**

In [97]:
#Guarda el año de la columana posted en una lista
#Si no hay un entero se guarda un Nan
lista=[]
for i in range(0,len(df)):
    try:
      lista.append(int(df.iloc[i][4][-5:-1]))
    except: 
       lista.append(pd.NA)

#Agrega una colunma con el año de posted
df['posted_year'] = lista

#Limpia el dataframe, ya que hay algunos valores de posted sin año
df.dropna(inplace=True)

#Se borra la columna posted, ya que solo se necesita el año
df.drop(['posted'], axis=1, inplace=True)

#### Transforma la columna **price** a float

In [98]:
#Hay strings dentro de la columna price
df['price'].unique()

array([19.99, 9.99, 6.99, 14.99, 24.99, 'Free to Play', 'Free To Play',
       29.99, 'Free', 39.99, 2.99, 4.99, 34.99, 17.99, 0.99, 3.99, 1.99,
       7.99, 11.99, 15.99, 59.99, 3.49, 12.99, 59.95, 13.98,
       'Play for Free!', 16.99, 8.99, 44.99, 5.99, 49.99, 69.99, 54.99,
       160.91, 0.49, 18.99, 23.96, 15.0, 13.99, 13.37, 'Third-party',
       79.99, 771.71, 139.92, 'Free Mod', 189.96, 1.87, 99.99, 21.99,
       32.99, 7.49, 'Install Now', 0.98, 12.89, 23.99,
       'Play WARMACHINE: Tactics Demo'], dtype=object)

In [99]:
#Crea una lista limpia sin string de la columna price, y colocar el valor de cero a los juegos gratis
lista=[]
for i in range(0,len(df)):
    if type(df.iloc[i][1]) == str: 
        if df.iloc[i][1] in ['Free to Play', 'Free To Play', 'Free', 'Play for Free!','Free Mod']:
           lista.append(float(0))#lista.append(float(0)) 
        else:
           lista.append(pd.NA)     
    else:
        lista.append(df.iloc[i][1])       


#Borra la columna price
df.drop(['price'], axis=1, inplace=True)  

#Crea la nueva columna price a partir de la lista
df['price'] = lista  

In [100]:
df['price'].unique()

array([19.99, 9.99, 6.99, 14.99, 24.99, 0.0, 29.99, 39.99, 2.99, 4.99,
       34.99, 17.99, 0.99, 3.99, 1.99, 7.99, 11.99, 15.99, 59.99, 3.49,
       12.99, 59.95, 13.98, 16.99, 8.99, 44.99, 5.99, 49.99, 69.99, 54.99,
       160.91, 0.49, 18.99, 23.96, 15.0, 13.99, 13.37, <NA>, 79.99,
       771.71, 139.92, 189.96, 1.87, 99.99, 21.99, 32.99, 7.49, 0.98,
       12.89, 23.99], dtype=object)

### Se crea un dataframe para la funcion 1 

In [6]:
#Se hace una copia de df_games
df1 = df_games.copy()


#### Se crea la columa **release_year** y se manipula la columna **price**

In [7]:
#Transforma a la columna release_date a datatime
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce')

#Crea una nueva columna con el valor de año de release_date
df1["release_year"] = (df1["release_date"].dt.strftime("%Y"))

#Cambia la columna release_year a int
df1.dropna(inplace=True)
df1["release_year"] = df1["release_year"].astype('int')#, errors="ignore")

#Borra la columna release_date, ya que no sera necesaria
df1.drop(['release_date'] , axis=1, inplace=True)


###########################################################################


#Crea una lista limpia sin string de la columna price, y colocar el valor de cero a los juegos gratis
lista=[]
for i in range(0,len(df1)):
    if type(df1.iloc[i][8]) == str: 
        if df1.iloc[i][8] in ['Free to Play', 'Free To Play', 'Free', 'Play for Free!','Free Mod']:
           lista.append(float(0))
        else:
           lista.append(pd.NA)     
    else:
        lista.append(df1.iloc[i][8])       


#Borra la columna price
df1.drop(['price'], axis=1, inplace=True)  

#Crea la nueva columna price a partir de la lista
df1['price'] = lista  

#### Se borar la columnas que no se necesitaran

In [8]:
df1.drop(['publisher', 'app_name', 'url',
       'reviews_url', 'specs', 'early_access'] , axis=1, inplace=True)

#### Cambia el nombre de la columna **release_year** a **Añio**

In [9]:
df1.rename(columns={'release_year': 'Añio'}, inplace=True)

## Se guarda el dataframe

### Se limpia el dataframe antes de guardar

In [105]:
#limpia los Nan y null
df.dropna(inplace=True)
df1.dropna(inplace=True)

#Reinicia los indices
df.reset_index(drop=True, inplace=True)
df1.reset_index(drop=True, inplace=True)

### Verificación del DataFrame

In [106]:
def verificar_tipo_datos(df):
    '''
    Realiza un análisis de los tipos de datos y la presencia de valores nulos en un DataFrame.

    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    los tipos de datos en cada columna, el porcentaje de valores no nulos y nulos, así como la
    cantidad de valores nulos por columna.

    Parameters:
        df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre_campo': Nombre de cada columna.
        - 'tipo_datos': Tipos de datos únicos presentes en cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.
    '''

    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
        
    return df_info

In [107]:
verificar_tipo_datos(df)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'list'>],100.0,0.0,0
1,developer,[<class 'str'>],100.0,0.0,0
2,user_id,[<class 'str'>],100.0,0.0,0
3,item_id_x,[<class 'str'>],100.0,0.0,0
4,recommend,[<class 'bool'>],100.0,0.0,0
5,review,[<class 'str'>],100.0,0.0,0
6,item_name,[<class 'str'>],100.0,0.0,0
7,playtime_forever,[<class 'float'>],100.0,0.0,0
8,release_year,[<class 'int'>],100.0,0.0,0
9,posted_year,[<class 'int'>],100.0,0.0,0


In [108]:
verificar_tipo_datos(df1)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,title,[<class 'str'>],100.0,0.0,0
1,id,[<class 'str'>],100.0,0.0,0
2,developer,[<class 'str'>],100.0,0.0,0
3,Añio,[<class 'int'>],100.0,0.0,0
4,price,[<class 'float'>],100.0,0.0,0


### Se guardan los dataframe transformados y limpios dentro de **\dataframes**

In [8]:
#Se guardan los dataframes en archivos .csv
df.to_csv(os.path.join('dataframe','df.csv'), index=False)
df1.to_csv(os.path.join('dataframe','df1.csv'), index=False)